In [2]:
# Colab setup ------------------
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade watermark"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
# ------------------------------

import numpy as np
import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

## Developing intuition: Steady state expression levels depend on protein production and removal rates.
Simple circuit, a single gene coding for a single protein expressed at a constant level

$\beta$: total rate of molecules per unit time.

$\gamma$: removal rate for degradation/dilution (protein removed through active degradation as well as dilution as cells grow and divide). It is proportional to $x$ because it refers to a process in which each protein molecule has a constant rate of removal – the more molecules are present, the more are removed in any given time interval.

#### Phenomenological modeling

$\gamma = \gamma_{degradation} + \gamma_{dilution}$

$\frac{dx}{dt}=\beta - \gamma x$

$\frac{dx}{dt}=\beta - \gamma x = 0$

$x = \frac{\beta}{\gamma}$

In [3]:
# Build theoretical curves for dimensionless r and beta
r = np.linspace(0, 20, 200)
beta = 1 / (1 + r)
init_slope = 1 - r

# Build plot
p = bokeh.plotting.figure(
    frame_height=225,
    frame_width=350,
    x_axis_label="r/Kd",
    y_axis_label="β(r)/β₀",
    x_range=[r[0], r[-1]],
    y_range=[0, 1],
)
p.line(r, init_slope, line_width=2, color="orange", legend_label="initial slope")
p.line(r, beta, line_width=2, legend_label="β(r)/β₀")
p.legend.click_policy = "hide"
p.legend[0].items = p.legend[0].items[::-1]

bokeh.io.show(p)


In [4]:
# Build the theoretical curves
r = np.linspace(0, 20, 200)
alpha_0 = .25
beta = alpha_0 + 1 / (1 + r)

# Build plot
p = bokeh.plotting.figure(
    frame_height=225,
    frame_width=350,
    x_axis_label="r/Kd",
    y_axis_label="β(r)/β₀",
    x_range=[r[0], r[-1]],
    y_range=[0, beta.max()],
)
p.line(
    [r[0], r[-1]],
    [alpha_0, alpha_0],
    line_width=2,
    color="orange",
    legend_label="basal (leaky) expression, α₀/β₀",
)
p.line(r, beta, line_width=2, legend_label="β(r)/β₀")
p.legend.click_policy = "hide"
p.legend[0].items = p.legend[0].items[::-1]

bokeh.io.show(p)

In [5]:
a = np.linspace(0, 20, 200)
beta_A = a / (1 + a)
beta_R = 1 / (1 + r)

# Build plot
p = bokeh.plotting.figure(
    frame_height=225,
    frame_width=350,
    x_axis_label="a/Kd, r/Kd",
    y_axis_label="β ∕ β₀",
    x_range=[r[0], r[-1]],
    y_range=[0, 1],
)

p.line(a, beta_A, line_width=2, legend_label="β(a)/β₀")
p.line(r, beta_R, line_width=2, color="tomato", legend_label="β(r)/β₀")
p.legend.location = "center_right"

p.output_backend = 'svg'

bokeh.io.show(p)

In [6]:
# Values of Hill coefficient
n_vals = [1, 2, 10, 25, np.inf]

# Compute activator response
x = np.concatenate((np.linspace(0, 1, 200), np.linspace(1, 4, 200)))
x_log = np.concatenate((np.logspace(-2, 0, 200), np.logspace(0, 2, 200)))

f_a = []
f_a_log = []
for n in n_vals:
    if n == np.inf:
        f_a.append(np.concatenate((np.zeros(200), np.ones(200))))
        f_a_log.append(f_a[-1])
    else:
        f_a.append(x ** n / (1 + x ** n))
        f_a_log.append(x_log ** n / (1 + x_log ** n))

# Repressor response
f_r = [1 - f for f in f_a]
f_r_log = [1 - f for f in f_a_log]

# Build plots
p_act = bokeh.plotting.figure(
    frame_height=225,
    frame_width=350,
    x_axis_label="x/k",
    y_axis_label="activating Hill function",
    x_range=[x[0], x[-1]],
)
p_rep = bokeh.plotting.figure(
    frame_height=225,
    frame_width=350,
    x_axis_label="x/k",
    y_axis_label="repressing Hill function",
    x_range=[x[0], x[-1]],
)

p_act_log = bokeh.plotting.figure(
    frame_height=225,
    frame_width=350,
    x_axis_label="x/k",
    y_axis_label="activating Hill function",
    x_range=[x_log[0], x_log[-1]],
    x_axis_type="log",
)
p_rep_log = bokeh.plotting.figure(
    frame_height=225,
    frame_width=350,
    x_axis_label="x/k",
    y_axis_label="repressing Hill function",
    x_range=[x_log[0], x_log[-1]],
    x_axis_type="log",
)

# Set up toggling between activating and repressing and log/linear
p_act.visible = True
p_rep.visible = False
p_act_log.visible = False
p_rep_log.visible = False

radio_button_group_ar = bokeh.models.RadioButtonGroup(
    labels=["activating", "repressing"], active=0, width=150
)
radio_button_group_log = bokeh.models.RadioButtonGroup(
    labels=["linear", "log"], active=0, width=100
)

col = bokeh.layouts.column(
    p_act,
    p_rep,
    p_act_log,
    p_rep_log,
    bokeh.models.Spacer(height=10),
    bokeh.layouts.row(
        bokeh.models.Spacer(width=55),
        radio_button_group_ar,
        bokeh.models.Spacer(width=75),
        radio_button_group_log,
    ),
)

radio_button_group_ar.js_on_change(
    "active",
    bokeh.models.CustomJS(
        args=dict(
            radio_button_group_log=radio_button_group_log,
            p_act=p_act,
            p_rep=p_rep,
            p_act_log=p_act_log,
            p_rep_log=p_rep_log,
        ),
        code="""
if (radio_button_group_log.active == 0) {
    if (p_act.visible == true) {
        p_act.visible = false;
        p_rep.visible = true;
    }
    else {
        p_act.visible = true;
        p_rep.visible = false;
    }
}
else {
    if (p_act_log.visible == true) {
        p_act_log.visible = false;
        p_rep_log.visible = true;
    }
    else {
        p_act_log.visible = true;
        p_rep_log.visible = false;
    }
}
""",
    ),
)

radio_button_group_log.js_on_change(
    "active",
    bokeh.models.CustomJS(
        args=dict(
            radio_button_group_ar=radio_button_group_ar,
            p_act=p_act,
            p_rep=p_rep,
            p_act_log=p_act_log,
            p_rep_log=p_rep_log,
        ),
        code="""
if (radio_button_group_ar.active == 0) {
    if (p_act_log.visible == true) {
        p_act_log.visible = false;
        p_act.visible = true;
    }
    else {
        p_act_log.visible = true;
        p_act.visible = false;
    }
}
else {
    if (p_rep_log.visible == true) {
        p_rep_log.visible = false;
        p_rep.visible = true;
    }
    else {
        p_rep_log.visible = true;
        p_rep.visible = false;
    }
}
""",
    ),
)

# Color scheme for plots
colors_act = bokeh.palettes.Blues256[32:-32][:: -192 // (len(n_vals) + 1)]
colors_rep = bokeh.palettes.Reds256[32:-32][:: -192 // (len(n_vals) + 1)]

# Populate glyphs
for i, n in enumerate(n_vals):
    legend_label = "n → ∞" if n == np.inf else f"n = {n}"
    p_act.line(x, f_a[i], line_width=2, color=colors_act[i], legend_label=legend_label)
    p_rep.line(x, f_r[i], line_width=2, color=colors_rep[i], legend_label=legend_label)
    p_act_log.line(
        x_log, f_a_log[i], line_width=2, color=colors_act[i], legend_label=legend_label
    )
    p_rep_log.line(
        x_log, f_r_log[i], line_width=2, color=colors_rep[i], legend_label=legend_label
    )

# Reposition legends
p_act.legend.location = "bottom_right"
p_act_log.legend.location = "bottom_right"

bokeh.io.show(col)

#### References

[Phenomenological model](https://en.wikipedia.org/wiki/Phenomenological_model)

[Models in biology: ‘accurate descriptions of our pathetic thinking’ (Gunawardena, 2014)](https://link.springer.com/article/10.1186/1741-7007-12-29#Sec5)